    HOT WIRE ANEMOMETER CALIBRATION v.3
    LEA - Wind Tunnel
    python version

Developer: Rafael C. F. Mendes

Date:29/06/2023   (v1: 24/11/2016)

Description: Use this program to move hot wire probe and taking data from CTA module.
             
Hardware: 
   - Arduino Uno (probe positioning system) with GRBL library (see more at https://github.com/grbl/grbl)
   - National Instrument - USB-4332 **Warning: Working voltage -10 to 10V**                 
   - Hot wire anemometer - miniCTA DANTEC

Before run it, install the following libraries:

    - pip install nidaqmx
    - pip install pyserial

In [93]:
#libraries

import nidaqmx          
import time
import numpy as np
import serial           
from datetime import datetime
from nidaqmx.constants import LineGrouping
from nidaqmx.constants import Edge
from nidaqmx.constants import AcquisitionType 
import os
from scipy.optimize import curve_fit


In [94]:
#Definitions
SampleRate=1000
AquisitionTimeInSeconds=5

# Define the serial port and baud rate
port = 'COM7'  # Update with your Arduino's port
baud_rate = 115200

In [95]:
# Function to call NI system 

def acquire_data(SampleRate, AquisitionTimeInSeconds):
    with nidaqmx.Task() as master_task:
        Sample=SampleRate*AquisitionTimeInSeconds
        master = master_task.ai_channels.add_ai_voltage_chan("Dev1/ai0")
        master_task.timing.cfg_samp_clk_timing(SampleRate, source='', active_edge=Edge.RISING, sample_mode=AcquisitionType.CONTINUOUS)
        master_task.stop()
        master_data = master_task.read(Sample, timeout=AquisitionTimeInSeconds+1.0)
    return master_data


In [96]:
# Create a serial object
ser = serial.Serial(port, baud_rate)

# Check if the serial port is open
if ser.is_open:
    print(f"Serial port {port} is open. Ready to communicate.")

ser.reset_input_buffer()
ser.reset_output_buffer()
ser.readline()

# Read data from Arduino
response = ser.readline().decode().strip()  # Read the response from Arduino
print("Response from Arduino:", response)

# Read data from Arduino
response = ser.readline().decode().strip()  # Read the response from Arduino
print("Response from Arduino:", response)

# Close the serial port
#ser.close()

# Send a command to Arduino
command = '$H\r\n'  # Example command
ser.write(command.encode())  # Encode the command and send it



Serial port COM7 is open. Ready to communicate.
Response from Arduino: Grbl 1.1f ['$' for help]
Response from Arduino: [MSG:'$H'|'$X' to unlock]


4

In [97]:
command = 'x-400 y-600\r\n'  # Example command
ser.write(command.encode())  # Encode the command and send it

response = ser.readline().decode().strip()  # Read the response from Arduino
print("Response from Arduino:", response)





Response from Arduino: ok


In [98]:
command = 'z-100\r\n'  # Example command
ser.write(command.encode())  # Encode the command and send it

response = ser.readline().decode().strip()  # Read the response from Arduino
print("Response from Arduino:", response)
ser.close()


Response from Arduino: ok


In [99]:

folder_name = "calibration"

if not os.path.exists(folder_name):
    # Create the folder
    os.mkdir(folder_name)

now = datetime.now()
date_string = now.strftime('%d-%b-%Y')
time_string = now.strftime('%H-%M-%S')

filename = os.path.join(folder_name,f"{date_string}_{time_string}.txt")


# See after the input function

In [100]:
i = 1
datap = 0
press = []
Usig = []

while datap >= 0:
    x = input('mmh2o: ')  
    datap = float(x)

    if datap >= 0:
        press.append(datap)
        data=acquire_data(SampleRate, AquisitionTimeInSeconds)
        Usig.append(np.mean(data))
        # Tsig(i) = np.mean(data[:, 2])  # Uncomment this line if needed

    i += 1
    if datap < 0:
        break
    
n = i - 1



ValueError: could not convert string to float: ''

In [ ]:


def compute_exponential_fit(Usig_c, U):
    # Define the exponential function
    def exponential_func(Usig_c, a, b):
        return (Usig_c ** 2 / (b - a)) ** (1 / 0.25)

    # Perform curve fitting
    popt, pcov = curve_fit(exponential_func, Usig_c, U)

    # Extract the computed coefficients a and b
    a, b = popt

    return a, b

In [ ]:
def compute_polynomial_fit(Usig_c, U):
    # Define the polynomial function
    def polynomial_func(Usig_c, p1, p2, p3, p4, p5):
        return p1 * Usig_c**4 + p2 * Usig_c**3 + p3 * Usig_c**2 + p4 * Usig_c + p5

    # Perform curve fitting
    popt, pcov = curve_fit(polynomial_func, Usig_c, U)

    # Extract the computed coefficients p1, p2, p3, p4, and p5
    p1, p2, p3, p4, p5 = popt

    return p1, p2, p3, p4, p5

In [ ]:

g=9.81
rhoH20=998
rhoAIR=1.12

Ec = 1  # sqrt((Twire-Treff)./(Twire-Temp))
Usig_c = Ec * Usig

U = np.sqrt(2 * rhoH20 * g * np.array(press) / 1000 / rhoAIR)

a, b = compute_exponential_fit(Usig_c, U)
p1, p2, p3, p4, p5 = compute_polynomial_fit(Usig_c, U)


#with open(filename, 'w') as file:
with open(filename, 'wt') as fid:
    fid.write('Coefficients kings law\t')
    fid.write('U(x) = (x^2/b-a)^(1/0.25)\n')
    fid.write('a \t b\n')
    fid.write(f'{a:12.4f} \t {b:12.4f}\n')

    fid.write('\nCoefficients poly4\t')
    fid.write('U(x) = p1*x^4 + p2*x^3 + p3*x^2 + p4*x + p5\n')
    fid.write('p1\t p2\t p3\t p4\t p5\n')
    fid.write(f'{p1:12.4f} \t {p2:12.4f} \t {p3:12.4f} \t {p4:12.4f} \t {p5:12.4f}\n')

    fid.write('\nSignal \t Pressure \t Velocity\n')
    for i in range(n-1):
        fid.write(f'{Usig_c[i]:12.8f} \t {press[i]:12.8f} \t {U[i]:12.8f}\n')


ValueError: `ydata` must not be empty!

In [ ]:
# Create a serial object
ser = serial.Serial(port, baud_rate)

# Check if the serial port is open
if ser.is_open:
    print(f"Serial port {port} is open. Ready to communicate.")

ser.reset_input_buffer()
ser.reset_output_buffer()
ser.readline()

# Read data from Arduino
response = ser.readline().decode().strip()  # Read the response from Arduino
print("Response from Arduino:", response)

# Read data from Arduino
response = ser.readline().decode().strip()  # Read the response from Arduino
print("Response from Arduino:", response)

# Close the serial port
#ser.close()

# Send a command to Arduino
command = '$H\r\n'  # Example command
ser.write(command.encode())  # Encode the command and send it

ser.close()


Serial port COM7 is open. Ready to communicate.
Response from Arduino: Grbl 1.1f ['$' for help]
Response from Arduino: [MSG:'$H'|'$X' to unlock]
